In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
print('done')

In [ ]:
prefix = 'https://www.cvedetails.com'
def cve_scraper(url, product, dataframe):
    #df = pd.DataFrame(columns=['product','cve_id'])
    #df = df.append({'product':product,'cve_id':''},ignore_index=True)
    #print(df)
    df = dataframe
    site = requests.get(url)
    soup = BeautifulSoup(site.text)
    #print(soup.prettify())

    list_of_cves = soup.find_all('tr', {'class': 'srrowns'})

    for tr in list_of_cves:
        table_elems = tr.find_all('td')
        for td in table_elems:
            if 'CVE' in td.text:
                df = df.append({'product':product,'cve_id':td.text},ignore_index=True)


    # for e in list_of_cves[0].find_all('td'):
    #     if 'CVE' in e.text:
    #         print(e.text)


    list_of_pages = soup.find('div', {'id': 'pagingb'})
    anchors = list_of_pages.find_all('a')

    for a in anchors:
        print(int(a.text))
        if(int(a.text) == 1):
            continue
        page_n = requests.get(prefix + a['href'])
        sp = BeautifulSoup(page_n.text)
        list_of_cves = sp.find_all('tr', {'class': 'srrowns'})
        for tr in list_of_cves:
            table_elems = tr.find_all('td')
            for td in table_elems:
                if 'CVE' in td.text:
                    df = df.append({'product':product,'cve_id':td.text},ignore_index=True)
    return df

In [ ]:
'''
The aim of this block is to collect information about product-specific CVEs, up to 2021 
'''
prefix = 'https://www.cvedetails.com'
df = pd.DataFrame(columns=['product','cve_id'])
url_products = 'https://www.cvedetails.com/top-50-products.php?year=0'
products_site = requests.get(url_products)
soup = BeautifulSoup(products_site.text)
#print(soup.prettify())
table = soup.find('table', {'class':'listtable'})
product,product_url = ('','')
for body in table:
    for tr in body:
        product,product_url = ('','')
        for td in tr:
            if 'Product Details' in str(td):
                product = td.text
            if 'All vulnerabilities related to products' in str(td):
                product_url = prefix + td['href']
        try:
            df = cve_scraper(product_url,product,df)
        except:
            pass

print(df)


In [77]:
print(df['product'].unique())

['']


In [ ]:
# Save to csv
df.to_csv(r'datasets/product_cveid.csv', index = False)

In [74]:
# Load from csv
product_cveid = pd.read_csv("datasets/product_cveid.csv")
product_cveid

,product,cve_id
0,NaN,CVE-2019-1000020
1,NaN,CVE-2019-1000019
2,NaN,CVE-2019-1000018
3,NaN,CVE-2019-17362
4,NaN,CVE-2019-16943
...,...,...
51243,NaN,CVE-1999-0022
51244,NaN,CVE-1999-0018
51245,NaN,CVE-1999-0009
51246,NaN,CVE-1999-0008


In [ ]:
'''
The aim of this block is to collect information about types of CVEs, up to 2021 
'''
url_type = 'https://www.cvedetails.com/vulnerabilities-by-types.php'